<a href="https://colab.research.google.com/github/Shona173/codes/blob/main/New_algorithm_with_heterogeneous_object.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
!pip install POT
!pip install trimesh
!pip install pyrender
!pip install ffmpeg-python
!pip install numpy trimesh scikit-image plyfile

In [73]:
import numpy as np
import numpy.random as random

import matplotlib.pyplot as plt
import skimage.measure
import trimesh
from plyfile import PlyData, PlyElement
import ffmpeg
import pyrender
import matplotlib.animation as animation
from google.colab import files
from matplotlib import animation
from scipy.ndimage import gaussian_filter
from IPython.display import HTML
import ot
import os
import time

In [70]:
!apt-get update -qq && apt-get install -y ffmpeg

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [50]:
def render_ply_sequence(ply_filenames, image_prefix="frame", save_images=True):
    scene = pyrender.Scene()
    viewer = pyrender.OffscreenRenderer(512, 512)
    for i, ply in enumerate(ply_filenames):
        mesh = trimesh.load(ply)
        scene.clear()
        scene.add(pyrender.Mesh.from_trimesh(mesh, smooth=False))
        color, _ = viewer.render(scene)
        if save_images:
            plt.imsave(f"{image_prefix}_{i:02d}.png", color)
    viewer.delete()
    for i, ply in enumerate(ply_filenames):
        mesh = trimesh.load(ply)
        scene.clear()
        scene.add(pyrender.Mesh.from_trimesh(mesh, smooth=False))
def render_ply_sequence(ply_filenames, image_prefix="frame", save_images=True):
    scene = pyrender.Scene()
    viewer = pyrender.OffscreenRenderer(512, 512)
    for i, ply in enumerate(ply_filenames):
        mesh = trimesh.load(ply)
        scene.clear()
        scene.add(pyrender.Mesh.from_trimesh(mesh, smooth=False))
        color, _ = viewer.render(scene)
        if save_images:
            plt.imsave(f"{image_prefix}_{i:02d}.png", color)
    viewer.delete()
    for i, ply in enumerate(ply_filenames):
        mesh = trimesh.load(ply)
        scene.clear()
        scene.add(pyrender.Mesh.from_trimesh(mesh, smooth=False))
        color, _ = viewer.render(scene)
        if save_images:
            plt.imsave(f"{image_prefix}_{i:02d}.png", color)
    viewer.delete()

In [12]:
def clamp(value, min_val, max_val):
    return np.maximum(min_val, np.minimum(value, max_val))

In [13]:
def sdf_to_distribution(f_sdf, grid, grid_size):
    vals = f_sdf(grid)
    mask = (vals < 0).astype(np.float32)
    normed = mask / np.sum(mask)
    return normed.reshape((grid_size, grid_size, grid_size))

In [14]:
def sdf_sphere(p, s=0.5):
    p = p.copy()
    return np.sqrt(np.sum(p**2, axis=1))-s

In [15]:
def sdf_box(p, b=[0.5,0.5,0.5]):
    p2 = p.copy()
    q = np.abs(p2) - b
    q2 = np.maximum(q, 0.0)
    q2 = np.linalg.norm(q2, axis=1, ord=2)
    q3 = np.minimum(np.maximum(q[:,0], np.maximum(q[:,1], q[:,2])), 0.0)
    return q2 + q3

In [16]:
def sdf_octahedron(p, s=1.0):
    p = np.abs(p)
    m = np.sum(p, axis=1) - s
    d = np.zeros(p.shape[0])

    # Get index for each branch
    idx_x = 3.0 * p[:, 0] < m
    idx_y = (~idx_x) & (3.0 * p[:, 1] < m)
    idx_z = (~idx_x) & (~idx_y) & (3.0 * p[:, 2] < m)
    idx_else = ~(idx_x | idx_y | idx_z)

    # Build q for each condition
    q = np.zeros_like(p)
    q[idx_x] = p[idx_x]
    q[idx_y] = p[idx_y][:, [1, 2, 0]]
    q[idx_z] = p[idx_z][:, [2, 0, 1]]

    # Calculate distance for q
    k = clamp(0.5 * (q[:, 2] - q[:, 1] + s), 0.0, s)
    d_tmp = np.linalg.norm(np.stack([q[:, 0], q[:, 1] - s + k, q[:, 2] - k], axis=1), axis=1)

    # Assign to d according to condition
    d[idx_x | idx_y | idx_z] = d_tmp[idx_x | idx_y | idx_z]
    d[idx_else] = m[idx_else] * 0.57735027  # 1/sqrt(3)

    return d

In [17]:
def color_points_inside_sdf(f_sdf, grid, base_color=np.array([1.0, 0.0, 0.0])):
    sdf_values = f_sdf(grid)
    inside_mask = sdf_values < 0
    colors = np.zeros((len(grid), 3))
    colors[inside_mask] = base_color
    return colors

In [18]:
def r_union(f1, f2):
    return f1 + f2 + np.sqrt(f1**2 + f2**2)

In [19]:
def r_intersection(f1,f2):
  return f1+f2-np.sqrt(f1**2+f2**2)

In [20]:
def inverse_distance_weighting(x, points, values, power=2, eps=1e-6):
    dists = np.linalg.norm(points - x, axis=1) + eps
    weights = 1.0 / (dists ** power)
    return np.sum(weights * values) / np.sum(weights)

In [21]:
def gen_grid(resolution, low=-1.0, high=1.0):
    idx = np.linspace(low, high, num=resolution)
    x, y, z = np.meshgrid(idx, idx,idx)
    V = np.concatenate((x.reshape((-1,1)), y.reshape((-1,1)),z.reshape((-1,1))), 1)

    return V

In [22]:
def export_volume_to_obj(volume,level, bounds, output_file):
    min_val = np.min(volume)
    max_val = np.max(volume)

    if max_val-min_val < 1e-6:
        print("Volume fluctuations are too small")
        return

    level = min_val + 0.5 * (max_val - min_val)

    try:
      verts, faces, normals, _ = skimage.measure.marching_cubes(volume, level=level)
    except RuntimeError as e:
        print("marching_cubes failed: {e}")
        return

    scale = (bounds[1] - bounds[0]) / (volume.shape[0] - 1)
    verts = verts * scale + bounds[0]

    mesh = trimesh.Trimesh(vertices=verts, faces=faces, vertex_normals=normals)
    mesh.export(output_file)

In [23]:
def conv_ot_barycenter(mu_list, alpha, K, niter=100, eps=1e-8):

    mu_list = [mu / np.sum(mu) for mu in mu_list]
    alpha = np.array(alpha, dtype=np.float32)
    alpha = alpha / np.sum(alpha)

    shape = mu_list[0].shape
    v_list = [np.ones(shape, dtype=np.float32) for _ in mu_list]
    bary = np.ones(shape, dtype=np.float32)

    for _ in range(niter):
        w_list = []
        d_list = []

        for i in range(len(mu_list)):
            KT_v = K(v_list[i])
            w = mu_list[i] / (KT_v + eps)
            d = v_list[i] * K(w)
            d = np.maximum(d, eps)
            w_list.append(w)
            d_list.append(d)

        log_d = np.stack([alpha[i] * np.log(d_list[i]) for i in range(len(alpha))], axis=0)
        bary = np.exp(np.sum(log_d, axis=0))

        for i in range(len(mu_list)):
            v_list[i] = v_list[i] * bary / (d_list[i] + eps)

    return bary

In [24]:
def color_mix(color1, color2, t):
    return (1 - t) * np.array(color1) + t * np.array(color2)

In [37]:
def spatial_dependent_color(grid, sdf_func, base_color):
    sdf_vals = sdf_func(grid)
    sdf_vals = np.clip(sdf_vals, -1.0, 1.0)
    weights = 0.5 * (1.0 - sdf_vals)  # -1 -> 1, +1 -> 0
    colors = np.outer(weights, base_color)
    return colors


In [57]:
def export_colored_morph_sdf(sdf1, sdf2, resolution=64, bounds=(-1.5, 1.5), ts=[0.0, 0.25, 0.5, 0.75, 1.0],
    color_start=[255, 0, 0], color_end=[0, 0, 255], output_prefix='morph'):

    for t in ts:
        alpha_list = [1 - t, t]
        sigma = 1.0

        x = np.linspace(bounds[0], bounds[1], resolution)
        X, Y, Z = np.meshgrid(x, x, x, indexing="ij")
        grid = np.stack([X.ravel(), Y.ravel(), Z.ravel()], axis=1)

        mu1 = sdf_to_distribution(lambda p: -sdf1(p), grid, resolution)
        mu2 = sdf_to_distribution(lambda p: -sdf2(p), grid, resolution)

        mu_bary = conv_ot_barycenter([mu1, mu2], alpha_list, lambda f: gaussian_filter(f, sigma=sigma), niter=100)
        min_mu = np.min(mu_bary)
        max_mu = np.max(mu_bary)
        level = min_mu + 0.5 * (max_mu - min_mu)
        verts, faces, normals, values = skimage.measure.marching_cubes(mu_bary, level)

        scale = (bounds[1] - bounds[0]) / (resolution - 1)
        verts = verts * scale + bounds[0]


        colors1 = spatial_dependent_color(grid, sdf1, np.array(color_start) / 255.0)
        colors2 = spatial_dependent_color(grid, sdf2, np.array(color_end) / 255.0)
        interpolated_colors = (1 - t) * colors1 + t * colors2

        grid_indices = np.clip((verts - bounds[0]) / scale, 0, resolution - 1).astype(int)
        flat_indices = (grid_indices[:, 0] * resolution * resolution +
                        grid_indices[:, 1] * resolution +
                        grid_indices[:, 2])
        vertex_colors = (interpolated_colors[flat_indices] * 255).astype(np.uint8)

        mesh = trimesh.Trimesh(vertices=verts, faces=faces, vertex_normals=normals, vertex_colors=vertex_colors)
        filename = f"{output_prefix}_{t:.2f}.ply"
        mesh.export(filename)
        print(f"Exported t={t:.2f} shape to: {filename}")

In [59]:
export_colored_morph_sdf(
    sdf1=sdf_sphere,
    sdf2=sdf_octahedron,
    resolution=64,
    bounds=(-1.5, 1.5),
    ts=[0.0, 0.25, 0.5, 0.75, 1.0],
    color_start=[255, 85, 70],
    color_end=[34, 20, 255],
    output_prefix="morph_obj"
)

Exported t=0.00 shape to: morph_obj_0.00.ply
Exported t=0.25 shape to: morph_obj_0.25.ply
Exported t=0.50 shape to: morph_obj_0.50.ply
Exported t=0.75 shape to: morph_obj_0.75.ply
Exported t=1.00 shape to: morph_obj_1.00.ply
